# Homeostasis à-la-SparseNet on the activation probability

In this notebook, we test the convergence of SparseNet as a function of different learning parameters. This shows the relative robusteness of this method according to the coding parameters, but also the importance of homeostasis to obtain an efficient set of filters.

See also :
* http://blog.invibe.net/posts/2015-05-05-reproducing-olshausens-classical-sparsenet.html for a description of how SparseNet is implemented in the scikit-learn package
* http://blog.invibe.net/posts/2015-05-06-reproducing-olshausens-classical-sparsenet-part-2.html for a descrtiption of how we managed to implement the homeostasis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import shl_scripts

In [4]:
from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
experiments = SHL_set({})

In [5]:
%%writefile {experiments.shl.data_cache}/{tag}.py

import numpy as np
np.set_printoptions(precision=2, suppress=True)

try:
    import sys
    command = sys.argv[1]
except:
    command = 'run'

from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
#opts = dict(homeo_method=tag, eta_homeo=0.01, alpha_homeo=0.7, verbose=0)
opts = dict(homeo_method=tag, verbose=0)

n_jobs = 10
n_jobs = 1
n_jobs = 4


experiments = SHL_set(opts, tag=tag)

variables = ['eta', 'alpha_homeo']
variables = ['eta', 'alpha_homeo']
variables = ['eta', 'eta_homeo', 'l0_sparseness', 'alpha_homeo', 'eta_homeo']

if command == 'run':
    experiments.run(variables=variables, n_jobs=n_jobs)

if command == 'plot':
    import matplotlib.pyplot as plt

    #fig, ax = experiments.scan(variable='eta', list_figures=[], display='dynamic')
    #fig, ax = experiments.scan(variable='eta', list_figures=[], display='final')
    list_figures = []
    list_figures = ['show_dico']

    display_variables = ['error', 'logL', 'perror', 'aerror', 'MI', 'cputime']
    display_variables = ['error', 'logL', 'perror', 'MC', 'cputime']
    for variable in variables:
        experiments = SHL_set(opts, tag=tag)
        experiments.scan(variable=variable, list_figures=list_figures, display='')
        plt.show()

        for display_variable in display_variables:
            print('Time evolution of ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='dynamic', display_variable=display_variable)
            plt.show()
        plt.show()

        for display_variable in display_variables:
            print('Comparison at the end of learning for ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='final', 
                                       label=tag, display_variable=display_variable)
            plt.show()
        

Overwriting data_cache/HAP.py


In [6]:
%run {experiments.shl.data_cache}/{tag}.py load

In [7]:
!ls -l {experiments.shl.data_cache}/{tag}*
!rm -fr {experiments.shl.data_cache}/{tag}*lock*

-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:47 data_cache/HAP - alpha_homeo=0.49443_dico.pkl_lock
-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:47 data_cache/HAP - alpha_homeo=0.49443_dico.pkl_lock_pid-2391_host-fortytwo
-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:48 data_cache/HAP - alpha_homeo=0.58045_dico.pkl_lock
-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:48 data_cache/HAP - alpha_homeo=0.58045_dico.pkl_lock_pid-2391_host-fortytwo
-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:48 data_cache/HAP - alpha_homeo=0.68144_dico.pkl_lock
-rw-r--r--  1 laurentperrinet  staff        0 May 11 16:48 data_cache/HAP - alpha_homeo=0.68144_dico.pkl_lock_pid-2391_host-fortytwo
-rw-r--r--  1 laurentperrinet  staff  1468079 May 11 15:26 data_cache/HAP - eta=0.00309_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  1468079 May 11 15:26 data_cache/HAP - eta=0.00363_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  1468079 May 11 15:26 data_cache/HAP - e

In [ ]:
%run {experiments.shl.data_cache}/{tag}.py run

[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Do

In [ ]:
%run {experiments.shl.data_cache}/{tag}.py plot

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts, pandas